In [28]:
%reload_ext autoreload
%autoreload 2

from mne.utils import set_log_level

import os.path as op
import pandas as pd
from hcp_central.s3_utils import download_from_s3_bucket, upload_to_s3, update_s3_file_tracking, get_aws_credentials,s3_glob
from hcp.io.file_mapping import get_s3_keys_meg, get_s3_keys_anatomy

set_log_level("warning")

In [17]:
ls /home/ubuntu/mne-hcp-data/*csv

/home/ubuntu/mne-hcp-data/hcp_restricted_data.csv
/home/ubuntu/mne-hcp-data/hcp_unrestricted_data.csv
/home/ubuntu/mne-hcp-data/motor_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/motor_source_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/rs_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/rs_source_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/storym_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/storym_source_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/unrel_subject_processing_map.csv
/home/ubuntu/mne-hcp-data/wm_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/wm_source_unrelated_subjects.csv


In [19]:
df = pd.read_csv(op.expanduser('~/mne-hcp-data/unrel_subject_processing_map.csv'))

In [20]:
subjects_rs = df.Subject[df.unrelated_rs_source].tolist()

In [24]:
aws_key, aws_secret = get_aws_credentials(
    op.join(op.expanduser("~"), "mne-hcp-aws", 'aws_hcp_details.csv'))

In [25]:
hcp_prefix = 's3://hcp-openaccess/HCP_900'
s3_keys = list()
max_runs = 3
run_inds = [0, 1, 2]
for subject in subjects_rs:
    hcp_data_types = [
        'rest',
    ]

    hcp_outputs = [
        'raw',
    ]

    # allow for downloading fewer data
    s3_keys = get_s3_keys_meg(
        subject,
        data_types=hcp_data_types,
        hcp_path_bucket='HCP_900',
        outputs=hcp_outputs,
        run_inds=run_inds[:max_runs])
    s3_keys = [kk for kk in s3_keys if 'config' in kk]

    download_from_s3_bucket(
        bucket='hcp-openaccess',
        out_path='/mnt/HCP', key_list=s3_keys, aws_key=aws_key,
        aws_secret=aws_secret,
        prefix='HCP_900')

Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:02
Elapsed time downloading hcp-openaccess from s3 00:00:01
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:01
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:01
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-op

In [46]:
aws_key_dyn, aws_secret_dyn = get_aws_credentials(
    op.join(op.expanduser("~"), "mne-hcp-aws", 'aws_details.csv'))

In [30]:
s3_glob?

In [58]:
hcp_prefix = 's3://hcp-openaccess/HCP_900'
s3_keys = list()
max_runs = 3
run_inds = [0, 1, 2]
for subject in subjects_rs:
    s3_keys = s3_glob(key_pattern='hcp-meg/%s/rest*raw.fif' % subject, bucket='hcp-meg-data',
         aws_key=aws_key_dyn, aws_secret=aws_secret_dyn, prefix='')
    download_from_s3_bucket(
        bucket='hcp-meg-data',
        out_path='/mnt', key_list=s3_keys,
        aws_key=aws_key_dyn,
        aws_secret=aws_secret_dyn,
        prefix='')
    break

Elapsed time downloading hcp-meg-data from s3 00:00:42
